In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import os

#os.chdir('../')
#sys.path.insert(0, '/Users/alanliu99/Documents/GitHub/yelp-recommender-system/src')
#from tfidf import *

In [2]:
#os.chdir('../')
#business = pd.read_csv('data/business.csv')
#review = pd.read_csv('test/testdata/test_review.csv')
las_vagas_reviews = pd.read_csv('data/Las_Vegas_review.csv')
reviews = pd.read_csv('data/reviews.csv')

In [22]:
business = pd.read_csv('data/business.csv')

In [23]:
cities = ['Las Vegas','Phoenix','Toronto']
business.dropna(inplace = True)
restaurants_business = business[business.categories.str.contains("Restaurants")]
business_city = restaurants_business[restaurants_business.city == 'Phoenix']
bus_rev_city = business_city.merge(reviews, on = 'business_id')
#bus_rev_city = bus_rev_city.drop(columns = ['Unnamed: 0'])

In [ ]:
larger_than_5_review_user = las_vegas_all_reviews[las_vegas_all_reviews.user_id.isin(filtered_larger_5.index.values)]
larger_than_5_review_user

### Preprocess for TF-IDF

In [ ]:
def preprocess_group_user_review(review_df):
    user_df = review_df[['text','categories','user_id']].groupby('user_id').agg({'text':list,
                                                                            'categories':lambda column: ' '.join(column)}
                                                                          ).reset_index()
    user_df.text = user_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return user_df

In [10]:
def preprocess_group_restaurant_review(review_df, business_df):
    restaurant_df = review_df[['text','stars','business_id']].groupby('business_id').agg({'text':list,
                                                                                          'stars':np.mean}
                                                                                        ).reset_index()
    restaurant_df.text = restaurant_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return restaurant_df.merge(business_df[['name', 'business_id']], on = 'business_id')

In [ ]:
joined_user = preprocess_group_user_review(las_vagas_reviews)
joined_user

In [25]:
joined_restaurant = preprocess_group_restaurant_review(bus_rev_city, business)
joined_restaurant

,business_id,text,stars,name
0,--g-a85VwrdZJNf0R95GcQ,Unhappy about the hours Change the yelp ad ...,4.434783,Kabab House
1,-050d_XIor1NpCuWkbIVaQ,This place is fantastic n nAbout the restau...,4.000000,Matt's Big Breakfast
2,-0aIra_B6iALlfqAriBSYA,Clean and fresh look atmosphere Don t love...,4.636364,Tea2go
3,-0tgMGl7D9B10YjSN2ujLA,Really enjoyed the crowd live music and fr...,3.380952,Dubliner
4,-1UMR00eXtwaeh59pEiDjA,Service sucks sooooooo bad I waited 25 minu...,3.320236,Matt's Big Breakfast
...,...,...,...,...
3292,zusXbyXnJfS1Q-pZE9elmQ,Not sure if this is a franchise store or com...,3.043478,Sonic Drive-In
3293,zvsJSNdg9UXEIL6FKiGa3Q,I give it one star customer service poor att...,1.692308,Taco Bell
3294,zwmps5SXn30g-f5wqg_r9A,WORST RESTAURANT I VE TRULY EVER BEEN TO ...,2.909091,Garcia's Mexican Restaurants
3295,zxVw9U56heInhfyprhkxIg,Food is reallyyy good I always get the che...,3.176471,La Paloma Mexican Food


## Filter out the grouped dataframe (optional)

In [ ]:
filtered_joined_restaurant = joined_restaurant[joined_restaurant.stars > 3]
filtered_joined_restaurant 

In [ ]:
filtered_joined_restaurant = joined_restaurant[joined_restaurant.stars > 3]
filtered_joined_restaurant 

### Convert the text review dataframe into the TF-IDF featue matrix

In [26]:
%%time
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.1, stop_words='english')
tfidf_matrix = tf.fit_transform(joined_restaurant.text.apply(lambda x: x.lower()))

CPU times: user 2min 25s, sys: 37.2 s, total: 3min 2s
Wall time: 3min 28s


In [ ]:
a = 'I prefer the fluffy toast at Canes though.'
b = 'We also had the BBQ Shrimp appetizer which wasn t what I expected BBQ Shrimp to be  but still really amazing'
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.1, stop_words='english')
tfidf_matrix = tf.fit_transform([a, b])

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
rest_rest_df = pd.DataFrame(cosine_sim, columns=joined_restaurant[0:5].name, index=joined_restaurant[0:5].name)
rest_rest_df

### User - User Simple TF-IDF and Cosine Similarity recommendation

In [ ]:
def user_recommendations(ID, df, record, k=10):
    """
    Recommends user based on a similarity dataframe
    Parameters
    ----------
    ID : str
        User ID (index of the similarity dataframe)
    df : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    record : pd.DataFrame
        The record of original user and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    ix = df.loc[:,ID].to_numpy().argpartition(range(-1,-k,-1))
    high_recomm = df.columns[ix[-1:-(k+2):-1]]
    high_recomm = high_recomm.drop(ID, errors='ignore')
    return pd.DataFrame(high_recomm).merge(record).head(k)

In [ ]:
# direct calculate the cosine similarity and generate the dataframe
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
user_user_df = pd.DataFrame(cosine_sim, columns=t.user_id, index=t.user_id)

In [ ]:
# given a user ID and the similarity matrix, recommend a list of similar user
user_id = '---1lKK3aKOuomHnwAkAow' # input user id
add_on = t[['user_id','text', 'categories']]

user_recommendations(user_id, user_user_df, add_on, k=5)

### Resraurant - Reataurant TF-IDF approach

In [ ]:
def restaurant_recommendations(business_ID, df, record, k=10):
    """
    Recommends user based on a similarity dataframe
    Parameters
    ----------
    ID : str
        Restaurant Business ID (index of the similarity dataframe)
    df : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    record : pd.DataFrame
        The record of original user and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    ix = df.loc[:,business_ID].to_numpy().argpartition(range(-1,-k,-1))
    high_recomm = df.columns[ix[-1:-(k+2):-1]]
    high_recomm = high_recomm.drop(business_ID, errors='ignore')
    return pd.DataFrame(high_recomm).merge(record).drop_duplicates().head(k)

In [27]:
# direct calculate the cosine similarity and generate the dataframe
cosine_sim = cosine_similarity(tfidf_matrix)

In [28]:
# direct calculate the cosine similarity and generate the dataframe
# cosine_sim = cosine_similarity(tfidf_matrix)
rest_rest_df = pd.DataFrame(cosine_sim, columns=joined_restaurant.business_id, index=joined_restaurant.business_id)

In [ ]:
# given a user ID and the similarity matrix, recommend a list of similar user
business_id = '4Nj2ktP2NjCbv2aKeH3ZBg' # input business id
add_on = las_vagas_reviews[['business_id','categories','name']].merge(filtered_joined_restaurant[['business_id','stars']])

restaurant_recommendations(business_id, rest_rest_df, add_on, k=15)

## Generate Top 100 nearest restaurant for each resraurant in Las Vegas

In [18]:
def generate_n_dataframe(df, n = 20):
    column = ['business_id']
    for i in range(n):
        column_name = 'Top_' + str(i)
        column.append(column_name)
    # top N recommended restaurant dataframe for all restaurants
    top_df = pd.DataFrame(columns = column)
    # iterate through all restaurants in the dataframe
    for i in df.index:
        ix = df.loc[:,i].to_numpy().argpartition(range(-1,-n,-1))
        high_recomm = df.columns[ix[-1:-(n+2):-1]]
        top_df = top_df.append(pd.Series(high_recomm.values, index = column),ignore_index=True)
    return top_df

In [29]:
result_df = generate_n_dataframe(rest_rest_df, n = 100)
result_df

,business_id,Top_0,Top_1,Top_2,Top_3,Top_4,Top_5,Top_6,Top_7,Top_8,...,Top_90,Top_91,Top_92,Top_93,Top_94,Top_95,Top_96,Top_97,Top_98,Top_99
0,--g-a85VwrdZJNf0R95GcQ,len2nrEtgD9bjnoYSyLPjA,ii_OxGUbtwd3S0RUW9OTvA,Er2XAYN8l1BadlYFwiPb9g,fgxW2Eww2MzMHd8eWT58Ig,ksZfCKQHYdCu0e8KbMuBDg,G8k2E3oYHHl2o4lNJQwR-g,Zp9RMGJKVc9Qkhj4QqyROw,OfLdWSqZ4HM_-B5khUtS8w,a6asAPtLHfWG871UBizACw,...,8RnOU6PLyOk3qHV1kZ4hcg,VgfqjXetLMxn5X5_uomBkg,1JF9TbJ2d5hH8xsQvvklHg,_lD9Y_0OfzZjfz5s-FVwqQ,hb96ToDVKWQkAj5h-UB5Ow,urkODQ1jX0qQzgbqsQjWVQ,YOD9dXrnpu8HTRILpF0onw,qms2KpC7A0KM3DxGqXWzaQ,R1k-z3Mauqk4v-7ZeEVsHQ,KFrJTt-NTXUrsdr7BGx5dg
1,-050d_XIor1NpCuWkbIVaQ,L2p0vO3fsS2LC6hhQo3CzA,24Td_CQH1bonWKff1rt2vg,8R_oemw4rNjJL6sd40O_oQ,utIA0LyQmwP-9DRyxUe6qQ,8Zqh2jwtncA3N4fWEMTvZQ,hH8002bWtOFYtLgarFZBsg,0go_co5nVtlbu4HgUyUIOQ,omU26JKzbqXhugm0O2U1mA,aMF8cG445ONPRKBsBYqofg,...,pu7maVMRHbIUv2x3B_xMHQ,JCZEK7wiazoM6xiq8YeZyw,2X07EuED0jY5C5hKQovfBA,V0GHF8CkhOovDeQKCn7YQg,Gra55w4glYeli9KmK_Id8w,nn4ZkIZTCd7jJhxs-P9tHA,02GctEso08hyhQpkqOftsw,uHZDCuzXrgWBKrGP7JVPmA,jsuUmIEefPjV__ads62Z5w,1Q_VjzXaq8Ad4rvSVOWpcg
2,-0aIra_B6iALlfqAriBSYA,BJRZ_7NSFU7xSrUQh1dVdQ,ogW-9p7lcXQYTPymXy9KXw,7Pjuyrj1xwRskC8RbUMdpg,_-OJ4-bPGyKQ9Hcc5JeSqw,I_a74zmgR-X03LsKISWPcg,qyOALwyMLbXKSt38glZ7yg,tGLz8WfLwSi221wgNh-0qw,bjrPt4DN1r4HpiR0Y8ofvg,yem1oUuZGl073Dlq_dMKIQ,...,fKm6mfATDJRDDQH0PtZeeQ,85o8XAZDoPmRqHFhdJLAGg,GVVf7BZ-4HiuKKbHaFTBsg,urkODQ1jX0qQzgbqsQjWVQ,-Y9woV5m-KaTzu9VpirYyg,5eK_pgro9_LxPYDoRVJnEA,KmlHgbdAXJ2OE5wV1vwVEw,roCxdzue-nME2akdIRPTfA,X1r8ugVdi4r6siqkNmIkkg,m506zjQ67j0msTfANwB6RA
3,-0tgMGl7D9B10YjSN2ujLA,rW5N-fiNhqb-fR3bGqyPdg,_sOni_t0FvAQuUDBEGe3Gg,V9bs-9SqhlryOCLy6vk6gg,UESdOMLsXn_7WcwOZt7I3A,kJhQq1BFz7lOYLve7xPaJQ,PbTh7aX0_yyaXFOXAlhI4A,EV1jAj7NFNmNIqOnwZ4Acw,zU63bs3Ofe8y1peOgOFJaA,avKtXuzalCP_cVfrk-fLeQ,...,82I9SSilxh3zL8F1UQ45Xw,ojI96R363pDCrdIlhVKTvQ,m1BRMnrel42nz_qIPSBO9w,aEs7Qle2Ah49dwaf3QNU1w,3QQEc5QUP-h6LCkUZQEYQw,VYuMUCoN6LWTxM80_itq1Q,R3B_ibZvlrgzch_UkYQkrg,aKFA85Miwb96A0IAS3mRgQ,VBoyqq5lO_H-EJO6MTwIPA,Uxj7dcad-xBDRR73jNEprA
4,-1UMR00eXtwaeh59pEiDjA,OeQUmob9q5sbEixK2-Tozw,xImiQIhC2ZBDEy7LhpQ4nQ,CzqYP9VKJBzGLnp14naScA,I3STZd5iAvUL9MNLF_DUug,L2p0vO3fsS2LC6hhQo3CzA,QTF2Vgd_I1uuS5398qdkQQ,ANHCo9X7zu0LdYecc9fk3Q,24Td_CQH1bonWKff1rt2vg,Ayt0hEO4siFH8h4lsCa4VQ,...,TUGeVyCHt_grCYs1Bzwnrg,mj_E2SThbggt-FyUaE1DQg,wB2mrl_xHyj55UObucaVRA,k4jX-Xe9dFu2pmdnlrcwgA,MJyKhCf7lu0XPhCbo-TnbQ,yzjQvTWiinB8pmbCMu8vuQ,up-RKqEfV9iIrtc6iyCDfw,EhK37wNBtzk64S8kksL9Og,n8RVrpWpcd3HprzEnJ3RxA,EIL41z-hvVCeYHqfA9PyWQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,zusXbyXnJfS1Q-pZE9elmQ,MOi64A9mDpK9zUB8546L8Q,IUacTdoWEWEODMVFlUOOlw,Yk8WiriuHOkAXufUqGZMyg,tI7dXg2UboJozj28v7sjLw,j7LCI60GYN1KxXRiy_LMkA,fvCiBVBsoY8ixMJ7Esk33w,HEmSK3apoP4xwVs0YkpNLQ,GdV24nwlR29MPqNXFglZnQ,Y26ixQb6rpJzdky0WdTjrA,...,LpdW1vh1AVm5QznEL5j03g,N5B7OBTyNGgwU_Y2fwNcyw,3kVThNUM1mzz1lBtxdSJxw,4hwe5w3opbrwFzezR6i7ig,2SHu52w_w4Z4ACq-vx3a-Q,h6neYN7OZVt9_ok2bAx6BA,FirWX-Ep5203TsdiGgShKg,w9gi2HA5a7asCwEF65GEWA,rPY4bukI1QdfJc-1OCAwLg,8jHr3atzErzItizENMkpzQ
3293,zvsJSNdg9UXEIL6FKiGa3Q,U7qvifH4kTy_Ct2dqb3KSg,nVGPxMooxSiaSH_wHvcRbw,qB15WElGAlI_eGWjn0kT2w,1DswS8btQKkWFyTUZlcgCQ,okX1jSK4-4uPdglVFNYhSw,-2isRNVb6PDuBagELL5EBw,8a4ScB2OYz8uVtp1PWP5jg,cCV0q8VV5RZIfcvolWXlBQ,V1CvqpVdyc1XkbQRPJqLCA,...,vf-OLMuP-EB1vlqVGUx4CQ,GUsYtGG557XKlHaMH87erg,nwHd2MbIGz0SNG8GpFjekw,v7UIeEvwNd3fleFE6icm3A,yHBhLawYmIthUPLbUXK5kw,HXdPqrO27tANiLpbCe9BVA,E_DXCeVllZtHV93hsAIZvw,ayaDm0WPZB0g0PYbGFkrzA,8jHr3atzErzItizENMkpzQ,X8OGDAf9F8HmjTqZ-OsvjA
3294,zwmps5SXn30g-f5wqg_r9A,Hgy5MrIKrwRJDXnwKaCpbA,ZCe93zAkxr2KOBU1NT0lmg,cKRMmytHxaSt8F0SMEzKqg,euL4wrPoczkOBkS93pxv5A,X6YpIqhaP_qoSZJRBcivGw,5KheTjYPu1HcQzQFtm4_vw,jAsvKe4-SteA-ZgyfVPoTQ,YM-nYIHd99qUt9n62AxIAg,zbrFk-4ejesAJD8EwcdHxg,...,j_prxgHnMvuRdrGjAI9qXQ,Veb_YSP2PL3xGbu9JJVuOA,uXoaycCmhc3Cg4Iei8kHYA,vyDW7_CHhDUJshMqaSjj0g,dGb8W42bi3ecRGK_I7pk0Q,EE_CKAAq7_oL7jfxmUKhuw,rCD-XFLBhscylZdg1OkCSQ,R1k-z3Mauqk4v-7ZeEVsHQ,3-aEgS7X2jrbxA7sA1nARw,UqhVZYUeoBGroYpZB-9jlQ
3295,zxVw9U56heInhfyprhkxIg,Sv1XtP

In [31]:
business

,name,business_id,city,categories,address,review_count,hours
0,The Range At Lake Norman,f9NumwFMBDn751xgFiRbNA,Cornelius,"Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...",10913 Bailey Rd,36,"{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
3,Nevada House of Hose,6OAZjbxqM5ol29BuHsil3w,North Las Vegas,"Hardware Stores, Home Services, Building Suppl...",1015 Sharp Cir,3,"{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,USE MY GUY SERVICES LLC,51M2Kk903DFYI6gnB5I6SQ,Mesa,"Home Services, Plumbing, Electricians, Handyma...",4827 E Downing Cir,26,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."
5,Oasis Auto Center - Gilbert,cKyLV5oWZJ2NudWgqs8VZw,Gilbert,"Auto Repair, Automotive, Oil Change Stations, ...","1720 W Elliot Rd, Ste 105",38,"{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ..."
6,Green World Cleaners,oiAlXZPIFm2nBCt0DHLu_Q,Las Vegas,"Dry Cleaning & Laundry, Local Services, Laundr...","6870 S Rainbow Blvd, Ste 117",81,"{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ..."
...,...,...,...,...,...,...,...
209384,Sunbelt Rentals,5d6Z-FjQ3g7TZxIs9mJ9kw,Tempe,"Building Supplies, Home Services",2066 E University Dr,5,"{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ..."
209385,Domino's,hskVqZCPqy-omm9CHi44xQ,Charlotte,"Restaurants, Pizza, Sandwiches, Fast Food, Chi...","6420 Rea Rd, Suite B1",16,"{'Monday': '10:30-0:0', 'Tuesday': '10:30-0:0'..."
209388,Nishi Sushi,9Q0fPWAjUweoFDk0kafuzQ,Vaughan,"Japanese, Sushi Bars, Restaurants",9750 Weston Road,5,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
209390,Five Guys,SYa2j1boLF8DcGVOYfHPcA,Scottsdale,"American (New), Food, Burgers, Restaurants, Fa...",7014-590 E Camelback Rd,97,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [32]:
k = joined_restaurant[['business_id','stars','name']].merge(result_df)
m = business[['business_id','categories','review_count','address']].merge(k).drop_duplicates()
l = m.reset_index(drop = True)

In [33]:
l

,business_id,categories,review_count,address,stars,name,Top_0,Top_1,Top_2,Top_3,...,Top_90,Top_91,Top_92,Top_93,Top_94,Top_95,Top_96,Top_97,Top_98,Top_99
0,vjTVxnsQEZ34XjYNS-XUpA,"Food, Pretzels, Bakeries, Fast Food, Restaurants",10,"4550 East Cactus Rd, #KSFC-4",3.800000,Wetzel's Pretzels,UT9KnZ288uwsz4pxzUVzIQ,_lD9Y_0OfzZjfz5s-FVwqQ,8NMf2dCmEGGKYR3SbMcnNA,rSasUpdyJO0sSC-UoOj38A,...,rnBffgr3QiXUpKef2YFScg,9GPWnFfwL3teZFax-CPD_A,xmAakatNKpvIJ-Fmq5swCw,ecn5dq7bFA6wCeeNnXxSzw,EUQoB_ph7_xIyj7JZGI3bg,TtbCOwOpNNzE7fW5CFwyng,VsHhy9ixEze-AxNYt6UxLA,2g4ZTD3ePNSbDAvh6qAOKQ,QsMR_Fp9YRtDegThsOgFLA,v2tjswIV3e0c8o7_7MwYmg
1,2nbibYLTR96Nr2nk-1spdw,"Restaurants, Fast Food, Chicken Wings, Chicken...",5,1546 E Roosevelt,3.400000,Church's Chicken,h_hg33vb0OsWsE6T1SHAgw,vVOoL5H8Fr-qlQv-_DdoMA,GWCd4xlTdOwcaaSSu9b9XQ,fYDzrVfx9uSbX7xmBVx-Hg,...,cTJjTKz2huGZ-ElScC2pSw,URa75T-A1QvjMUFNIPyukQ,tDYcVluqZwieulc1iqxGXg,sO2kZPPIlmp-c-APO7xCnw,YKImxc9VimLbDl2sQN1BBQ,3aKmNE2coy5YLBCIu676og,cz5vz-893D3LNH3TMQYR1w,YiuI4YHJnrl9898gE50_ow,LypV5tUYSDurg_VMHVLpDw,R9i4v1iPBCFeqzMkQEcfjQ
2,W-5cdzVAeRv0EtAeed4tqw,"Hot Dogs, Fast Food, Restaurants, Middle Eastern",9,8941 N Central Ave,3.800000,Sam's Gyros,aw0H9O1yysoHB0JanT0Ubw,p_-fGCrpUQZXSO6sGAs6Tg,ylxqmxh2gO1yCpQkIk6o3A,Qng9lEqWA9uTtm6oq2rZuw,...,DGXbfl2cE8MjkjTac2r3xA,SWDxTrwU_qlEcJp36IYquw,2g4ZTD3ePNSbDAvh6qAOKQ,aKFA85Miwb96A0IAS3mRgQ,ZZxgrXYDC6RIPrWCl9kdjA,Jj8ubiwwuCR-rrhrrjcryw,_rWuWqU8GHLqSdsoa4BPrw,i3MWkfRgwqPYMKyL9Mk_Pg,F-ULgBFmiSqG9udQmu6CCw,3aKmNE2coy5YLBCIu676og
3,eCJDiRPoSuA0bsb7F1gNUA,"Fast Food, Chinese, Szechuan, Asian Fusion, Re...",49,"2510 W Thunderbird Rd, Ste 9",3.897959,Asian Cafe,pdy6EAcJvZks1hKogxOj6Q,UUtWoDyREiy9MPdtTRFUYQ,nBwOIq5KI-zmxp061dTmgw,R7-3MCD6_RL_VxN65yCh2Q,...,E8ugJa02FVCgfnjqV817iw,kBJfBVaXJtPEKLTov1hJdQ,ZIdR-IopAtU5PH_mGtUAbw,C5bmtDRAtl5iMGmBCF5prw,KWTv8elo9HiN9sIKvN0_bQ,YZQVfJOprZIN6eTc8q_C7w,vn5Ebwy-IdtJ6PQE6y7vaA,q4i-kcryM2X8VXWQcGbiSw,baIT89GubjGJV1mpn82Eeg,RnxOmwWVsJC87ztJPrHsig
4,a_E-rLe2zcyX-X4FKhWWRg,"Restaurants, Mexican",14,2808 W Mcdowell Rd,2.928571,Tacos y Mariscos Sinaloa,ibOX3CypYVz0nJhCN5Wmcw,Sv1XtPN-utoenqdIn9ZMyQ,TEaBolNAkOdRm2gvIYz4OQ,2xMKHQP1TVGB-_62gDOzDg,...,G_mqw9nNYDFkwtCe5Jd8QA,R1k-z3Mauqk4v-7ZeEVsHQ,1JF9TbJ2d5hH8xsQvvklHg,5KheTjYPu1HcQzQFtm4_vw,9d7N-I00hu-VkXFry4Da-g,bR5Pwh9sgTqThaHwpFXdaQ,fXUJ9d5R6kNXrnAouByB6g,joTw5858fDh3Y_rF0NDOsg,8Syk4yZRivDlVUdXxIPutQ,RdK6dhy4lOb2taNp-WrHjQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,9umA26_aWjEuVe4QW7axtw,"Chicken Wings, Italian, Salad, Restaurants, Pizza",126,4001 East Bell Rd,4.507246,Paradise Valley Pizza Company,OASX1Yrgq_7R1PsYpixRcw,_MHUEErH3KW_KvVSOjyYhg,ErHMVkPstNyefb3ZU_z1Cw,XF4w6wG1JOvfKZlQcxiWgQ,...,lwpQQflP1CGJPV7QLybzOg,wxzG81ZyWpBje_mU4aFLaw,VsHhy9ixEze-AxNYt6UxLA,kacZeom987POja3RwNUJpA,S3RKq5WDrvUO5ss6I2gNVQ,e-gVqGEtnjEkunvQ5J3uyA,AtZuw9MsU1lTdWo3DJAzWA,8xyRFdmF_9boBaPx0jBhHQ,cs1qJvnWUg_f8g020z_3PQ,pl0KAXaQbYN6B2CV_Y3vnw
3293,OE1emTl3wuvxy01ufROBtw,"Restaurants, Steakhouses",25,"The Shops at the Esplanade, 2501 E. Camelback Rd.",4.037037,Morton's The Steakhouse,7jl9EJQCGBiTyXTDXMsRIw,LUDX--wfStrKavGyitk4nA,_DPJ_PHJnXsuhYhsgBA0Xw,jZ8ogksrNzUmREJggc1YOQ,...,pKiSzUlBBRNEdTML24piXA,f30-DOoIxvoV9btiDvKMTw,BvTU8s3v-qfTtsSKV6Y16A,xdD3EXvF_p9WUiwwKpol_w,OZqAZkjhV_nyHsbnKaD1WQ,flE7Sn2JGs0n3SAtaep6Zw,cQNGkOyCvnIqs78vO8SGJQ,VEe-_Yc1fbEtUjkACEjLpA,fzxvmA3Ygu3yrQQ6-cEpTw,Aqy2wQmWH1YkiCaQeqG1jg
3294,O3fzOdRgcd5UFjymimyJjQ,"Food, Ice Cream & Frozen Yogurt, Burgers, Rest...",32,2734 W Camelback Rd,2.781250,Dairy Queen,h4U3h1RbgLvHl0fKSSUhPA,UUyAGY5-6SbMfS7Iun5AGg,BWVMaAZS6DMVk_wU9jVc7w,k1bwHg1izaXnUz67pSx_zQ,...,5DDumAHRKumv3gZiPKW6iw,KOA3EA_eFxiRz5P2lAmM_Q,LoCnjxSpQBoFLNWAmbmTqw,bJeVUZ8vCVLGYGIaVbY4Xw,AF-N6i0XCJRPFFOZKp1Pnw,YWjfWHizWgHSsAPGfQChXw,V0GHF8CkhOovDeQKCn7YQg,vyDW7_CHhDUJshMqaSjj0g,r9J4uFdWhDgH8poy1KTdDg,Y26ixQb6rpJzdky0WdTjrA
3295,wTKMQD3nOwEKGZmtiiiGXA,"Restaurants, Breakfast & Brunch, Sandwiches, A...",63,"101 N 1st Ave, Ste 108",3.701493,SpoonZ Cafe,f

In [34]:
l.to_csv('PX_100_rest.csv', index = False)

In [ ]:
l = m.reset_index(drop = True)
l

### Other Try

In [ ]:
for i in tqdm(range(len(cities))):
    business_city = restaurants_business[restaurants_business.city == cities[i]]
    bus_rev_city = business_city.merge(reviews, on = 'business_id')

In [ ]:
def calc_sim_user_rating(target_user_id, target_business_id, groupby_df, original_df):
    business_id_df = original_df[original_df.business_id == target_business_id].copy()
    
    target_user_tfidf = tfidf_matrix[groupby_df[groupby_df.user_id == target_user_id].index[0]].copy()
    
    dist_user = []
    for user_id in business_id_df.user_id.values:
        dist_user.append(cosine_similarity(tfidf_matrix[groupby_df[groupby_df.user_id==user_id].index[0]],
                                           target_user_vector)[0][0])
        
    business_id_df['similarity'] = dist_user    
    #Filtering technique = Calc percentile and filter percentiles > 80%
    business_id_df = business_id_df[business_id_df.rank(pct=True).similarity > .8]
     
    rating_diff = []
    for user_id in business_id_df.user_id.values:
        rating_diff.append(test_rest_user_id.stars_review[test_rest_user_id.user_id==user_id].values[0]
                            -np.average(city_train[city_train.user_id ==user_id].stars_review.values))
    test_rest_user_id['rating_diff'] = rating_diff
    
    return np.average(test_rest_user_id.rating_diff)

In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in test.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], test['user_id'][i]) for i in similar_indices]

    results[row['user_id']] = similar_items[1:]
    
print('done!')

def item(id):
    return test.loc[test['user_id'] == id]['text'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

In [35]:
score_reviews = pd.read_csv('data/PX_rest_info.csv')
score_reviews

,business_id,categories,review_count,address,stars,name,Top_0,Top_1,Top_2,Top_3,...,Top_90,Top_91,Top_92,Top_93,Top_94,Top_95,Top_96,Top_97,Top_98,Top_99
0,vjTVxnsQEZ34XjYNS-XUpA,"Food, Pretzels, Bakeries, Fast Food, Restaurants",10,"4550 East Cactus Rd, #KSFC-4",3.800000,Wetzel's Pretzels,UT9KnZ288uwsz4pxzUVzIQ,_lD9Y_0OfzZjfz5s-FVwqQ,8NMf2dCmEGGKYR3SbMcnNA,rSasUpdyJO0sSC-UoOj38A,...,rnBffgr3QiXUpKef2YFScg,9GPWnFfwL3teZFax-CPD_A,xmAakatNKpvIJ-Fmq5swCw,ecn5dq7bFA6wCeeNnXxSzw,EUQoB_ph7_xIyj7JZGI3bg,TtbCOwOpNNzE7fW5CFwyng,VsHhy9ixEze-AxNYt6UxLA,2g4ZTD3ePNSbDAvh6qAOKQ,QsMR_Fp9YRtDegThsOgFLA,v2tjswIV3e0c8o7_7MwYmg
1,2nbibYLTR96Nr2nk-1spdw,"Restaurants, Fast Food, Chicken Wings, Chicken...",5,1546 E Roosevelt,3.400000,Church's Chicken,h_hg33vb0OsWsE6T1SHAgw,vVOoL5H8Fr-qlQv-_DdoMA,GWCd4xlTdOwcaaSSu9b9XQ,fYDzrVfx9uSbX7xmBVx-Hg,...,cTJjTKz2huGZ-ElScC2pSw,URa75T-A1QvjMUFNIPyukQ,tDYcVluqZwieulc1iqxGXg,sO2kZPPIlmp-c-APO7xCnw,YKImxc9VimLbDl2sQN1BBQ,3aKmNE2coy5YLBCIu676og,cz5vz-893D3LNH3TMQYR1w,YiuI4YHJnrl9898gE50_ow,LypV5tUYSDurg_VMHVLpDw,R9i4v1iPBCFeqzMkQEcfjQ
2,W-5cdzVAeRv0EtAeed4tqw,"Hot Dogs, Fast Food, Restaurants, Middle Eastern",9,8941 N Central Ave,3.800000,Sam's Gyros,aw0H9O1yysoHB0JanT0Ubw,p_-fGCrpUQZXSO6sGAs6Tg,ylxqmxh2gO1yCpQkIk6o3A,Qng9lEqWA9uTtm6oq2rZuw,...,DGXbfl2cE8MjkjTac2r3xA,SWDxTrwU_qlEcJp36IYquw,2g4ZTD3ePNSbDAvh6qAOKQ,aKFA85Miwb96A0IAS3mRgQ,ZZxgrXYDC6RIPrWCl9kdjA,Jj8ubiwwuCR-rrhrrjcryw,_rWuWqU8GHLqSdsoa4BPrw,i3MWkfRgwqPYMKyL9Mk_Pg,F-ULgBFmiSqG9udQmu6CCw,3aKmNE2coy5YLBCIu676og
3,eCJDiRPoSuA0bsb7F1gNUA,"Fast Food, Chinese, Szechuan, Asian Fusion, Re...",49,"2510 W Thunderbird Rd, Ste 9",3.897959,Asian Cafe,pdy6EAcJvZks1hKogxOj6Q,UUtWoDyREiy9MPdtTRFUYQ,nBwOIq5KI-zmxp061dTmgw,R7-3MCD6_RL_VxN65yCh2Q,...,E8ugJa02FVCgfnjqV817iw,kBJfBVaXJtPEKLTov1hJdQ,ZIdR-IopAtU5PH_mGtUAbw,C5bmtDRAtl5iMGmBCF5prw,KWTv8elo9HiN9sIKvN0_bQ,YZQVfJOprZIN6eTc8q_C7w,vn5Ebwy-IdtJ6PQE6y7vaA,q4i-kcryM2X8VXWQcGbiSw,baIT89GubjGJV1mpn82Eeg,RnxOmwWVsJC87ztJPrHsig
4,a_E-rLe2zcyX-X4FKhWWRg,"Restaurants, Mexican",14,2808 W Mcdowell Rd,2.928571,Tacos y Mariscos Sinaloa,ibOX3CypYVz0nJhCN5Wmcw,Sv1XtPN-utoenqdIn9ZMyQ,TEaBolNAkOdRm2gvIYz4OQ,2xMKHQP1TVGB-_62gDOzDg,...,G_mqw9nNYDFkwtCe5Jd8QA,R1k-z3Mauqk4v-7ZeEVsHQ,1JF9TbJ2d5hH8xsQvvklHg,5KheTjYPu1HcQzQFtm4_vw,9d7N-I00hu-VkXFry4Da-g,bR5Pwh9sgTqThaHwpFXdaQ,fXUJ9d5R6kNXrnAouByB6g,joTw5858fDh3Y_rF0NDOsg,8Syk4yZRivDlVUdXxIPutQ,RdK6dhy4lOb2taNp-WrHjQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,9umA26_aWjEuVe4QW7axtw,"Chicken Wings, Italian, Salad, Restaurants, Pizza",126,4001 East Bell Rd,4.507246,Paradise Valley Pizza Company,OASX1Yrgq_7R1PsYpixRcw,_MHUEErH3KW_KvVSOjyYhg,ErHMVkPstNyefb3ZU_z1Cw,XF4w6wG1JOvfKZlQcxiWgQ,...,lwpQQflP1CGJPV7QLybzOg,wxzG81ZyWpBje_mU4aFLaw,VsHhy9ixEze-AxNYt6UxLA,kacZeom987POja3RwNUJpA,S3RKq5WDrvUO5ss6I2gNVQ,e-gVqGEtnjEkunvQ5J3uyA,AtZuw9MsU1lTdWo3DJAzWA,8xyRFdmF_9boBaPx0jBhHQ,cs1qJvnWUg_f8g020z_3PQ,pl0KAXaQbYN6B2CV_Y3vnw
3293,OE1emTl3wuvxy01ufROBtw,"Restaurants, Steakhouses",25,"The Shops at the Esplanade, 2501 E. Camelback Rd.",4.037037,Morton's The Steakhouse,7jl9EJQCGBiTyXTDXMsRIw,LUDX--wfStrKavGyitk4nA,_DPJ_PHJnXsuhYhsgBA0Xw,jZ8ogksrNzUmREJggc1YOQ,...,pKiSzUlBBRNEdTML24piXA,f30-DOoIxvoV9btiDvKMTw,BvTU8s3v-qfTtsSKV6Y16A,xdD3EXvF_p9WUiwwKpol_w,OZqAZkjhV_nyHsbnKaD1WQ,flE7Sn2JGs0n3SAtaep6Zw,cQNGkOyCvnIqs78vO8SGJQ,VEe-_Yc1fbEtUjkACEjLpA,fzxvmA3Ygu3yrQQ6-cEpTw,Aqy2wQmWH1YkiCaQeqG1jg
3294,O3fzOdRgcd5UFjymimyJjQ,"Food, Ice Cream & Frozen Yogurt, Burgers, Rest...",32,2734 W Camelback Rd,2.781250,Dairy Queen,h4U3h1RbgLvHl0fKSSUhPA,UUyAGY5-6SbMfS7Iun5AGg,BWVMaAZS6DMVk_wU9jVc7w,k1bwHg1izaXnUz67pSx_zQ,...,5DDumAHRKumv3gZiPKW6iw,KOA3EA_eFxiRz5P2lAmM_Q,LoCnjxSpQBoFLNWAmbmTqw,bJeVUZ8vCVLGYGIaVbY4Xw,AF-N6i0XCJRPFFOZKp1Pnw,YWjfWHizWgHSsAPGfQChXw,V0GHF8CkhOovDeQKCn7YQg,vyDW7_CHhDUJshMqaSjj0g,r9J4uFdWhDgH8poy1KTdDg,Y26ixQb6rpJzdky0WdTjrA
3295,wTKMQD3nOwEKGZmtiiiGXA,"Restaurants, Breakfast & Brunch, Sandwiches, A...",63,"101 N 1st Ave, Ste 108",3.701493,SpoonZ Cafe,f

In [66]:
text = "SpoonZ Cafe"
recomm_rest = []
target_column = ['name','address','stars','review_count','categories']
for i in range(5):
    rank = 'Top_' + str(i)
    target_id = score_reviews[score_reviews.name == text][rank].value_counts().idxmax()
    recomm_rest.append(score_reviews[score_reviews.business_id == target_id][target_column].values.tolist()[0])

In [67]:
business[business.name == "Donovan's Steak & Chop House"]

,name,business_id,city,categories,address,review_count,hours
150929,Donovan's Steak & Chop House,7jl9EJQCGBiTyXTDXMsRIw,Phoenix,"Bars, Restaurants, Nightlife, Seafood, Steakho...",3101 E Camelback Rd,343,"{'Monday': '0:0-0:0', 'Tuesday': '16:0-22:0', ..."


In [68]:
recomm_rest

[['Chestnut Fine Foods & Provisions',
  '4350 E Camelback Rd, Bldg I-100',
  3.369774919614148,
  301,
  'Cafes, Breakfast & Brunch, Ramen, Restaurants, American (New), Japanese'],
 ['Hob Nobs',
  '149 W McDowell Rd',
  3.82183908045977,
  335,
  'Local Flavor, Restaurants, Cafes, Sandwiches, Pizza, American (New), Delis, Arts & Entertainment, Breakfast & Brunch, Food, Nightlife, Jazz & Blues, Coffee & Tea'],
 ["Ollie Vaughn's",
  '1526 E McDowell Rd',
  4.43,
  297,
  'Food, Salad, Cafes, Coffee & Tea, Restaurants, Sandwiches, Bakeries, Breakfast & Brunch'],
 ['Essence Bakery Cafe',
  '3830 E Indian School Rd',
  3.956656346749226,
  307,
  'Coffee & Tea, Sandwiches, Bakeries, Food, Breakfast & Brunch, Restaurants'],
 ['EarlyBaker',
  '4025 E Chandler Blvd',
  4.501597444089457,
  299,
  'Custom Cakes, Bakeries, American (New), Food, Patisserie/Cake Shop, Mediterranean, Cafes, Greek, Breakfast & Brunch, Restaurants']]